In [26]:
import feedparser
import csv
import pandas as pd
import re
import numpy as np

Укажите словарь с RSS-лентой, обратите внимание, что не все RSS-ленты имеют доступный для парсинга формат, проверяйте
<br/>Укажите два вектора по которым будет производить поиск, символ '|' означает "или", если нужен только один вектор, в конце другого добавьте '|' - это будет означать, что вектор ищет все слова. Векторы не чувствительны к регистрам
<br/>Укажите два пути для файлов, для всех новостей и для определенных новостей

In [27]:
newsurls = {'Lenta.ru': 'https://lenta.ru/rss/',
'Vesti': 'https://www.vesti.ru/vesti.rss',
'RBC': 'http://static.feed.rbc.ru/rbc/logical/footer/news.rss',
'1CBUH': 'https://buh.ru/rss/?chanel=news',
'Regnum': 'https://regnum.ru/rss/news',
'Lenta': 'https://lenta.ru/rss',
'RT': 'https://www.rt.com/rss/',
'VZGLYAD': 'https://vz.ru/rss.xml',
'RG': 'https://rg.ru/xml/index.xml',
'Interfax': 'https://www.interfax.ru/rss.asp',
'NezGaz': 'https://www.ng.ru/rss/',
'MK': 'https://www.mk.ru/rss/index.xml',
'Klerk.ru': 'https://www.klerk.ru/export/news.rss',
'RT': 'https://www.rt.com/rss/'}

f_all_news = 'allnews23march.csv'
f_certain_news = 'certainnews23march.csv'

vector1 = 'Крым' # в векторах можно указывать сколько угодно слов, но если хочешь, чтобы искало по одному слову,
                 # в конце второго вектора надо ставить | (или)
vector2 = 'Деньги|'

In [28]:
def parseRSS( rss_url ): #функция получает линк на рсс ленту, возвращает распаршенную ленту с помощью feedpaeser
    return feedparser.parse(rss_url)
    
def getHeadlines(rss_url): #функция для получения заголовков новости
    headlines = []
    feed = parseRSS(rss_url)
    for newsitem in feed['items']:
        headlines.append(newsitem['title'])
    return headlines

def getDescriptions(rss_url): #функция для получения описания новости
    descriptions = []
    feed = parseRSS(rss_url)
    for newsitem in feed['items']:
        descriptions.append(newsitem['description'])
    return descriptions

def getLinks(rss_url): #функция для получения ссылки на источник новости
    links = []
    feed = parseRSS(rss_url)
    for newsitem in feed['items']:
        links.append(newsitem['link'])
    return links

def getDates(rss_url): #функция для получения даты публикации новости
    dates = []
    feed = parseRSS(rss_url)
    for newsitem in feed['items']:
        dates.append(newsitem['published'])
    return dates

In [29]:
allheadlines = []
alldescriptions = []
alllinks = []
alldates = []
# Прогоняем нашии URL и добавляем их в наши пустые списки
for key,url in newsurls.items():
    allheadlines.extend(getHeadlines(url))
    
for key,url in newsurls.items():
    alldescriptions.extend(getDescriptions(url))
    
for key,url in newsurls.items():
    alllinks.extend(getLinks(url))
    
for key,url in newsurls.items():
    alldates.extend(getDates(url))

In [30]:
def write_all_news(all_news_filepath): #функция для записи всех новостей в .csv, возвращает нам этот датасет
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
            
    return df

def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #функция для поиска, а затем записи
                                                                                         #определенных новостей по таргета,
                                                                                        #затем возвращает этот датасет
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

In [31]:
write_all_news(f_all_news) #все новости

,Title,Description,Links,Publication Date
0,Минобороны заявило об отражении атаки ВСУ на н...,Вооруженные силы Украины (ВСУ) на николаево-кр...,https://lenta.ru/news/2022/10/08/minobrrrr/,"Sat, 08 Oct 2022 15:37:00 +0300"
1,Глава Минтранса назвал сроки запуска движения ...,Глава Минтранса России Виталий Савельев сообщи...,https://lenta.ru/news/2022/10/08/srokizapuska/,"Sat, 08 Oct 2022 15:37:00 +0300"
2,Пять поездов из Крыма и Севастополя пообещали ...,"Оператор пассажирских перевозок в Крым, компан...",https://lenta.ru/news/2022/10/08/sevas/,"Sat, 08 Oct 2022 15:35:00 +0300"
3,ВКС России уничтожили более 100 украинских вое...,ВКС России разбили две бригады ВСУ на территор...,https://lenta.ru/news/2022/10/08/100_voenn/,"Sat, 08 Oct 2022 15:33:00 +0300"
4,Пентагон пообещал Украине новое оружие,Пентагон поставит украинской армии зенитно-рак...,https://lenta.ru/news/2022/10/08/pentagon_weapon/,"Sat, 08 Oct 2022 15:31:00 +0300"
...,...,...,...,...
1120,Бюджетные организации могут принимать строител...,Первичка для бюджетников утверждена приказом М...,https://www.klerk.ru/buh/news/537063/,"Fri, 07 Oct 2022 10:19:00 +0300"
1121,Банкротов среди россиян стало в два раза меньш...,За девять месяцев 2022 года рост количества ба...,https://www.klerk.ru/buh/news/537054/,"Fri, 07 Oct 2022 10:10:00 +0300"
1122,В новые российские регионы в течение года прид...,"Сначала в ДНР, ЛНР, Запорожской и Херсонской о...",https://www.klerk.ru/buh/news/537057/,"Fri, 07 Oct 2022 09:36:00 +0300"
1123,В России растет число финансовых пирамид. Но о...,ЦБ выявил почти 1500 компаний с признаками фин...,https://www.klerk.ru/buh/news/537052/,"Fri, 07 Oct 2022 08:59:00 +0300"


In [32]:
looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) #новости по вектору

,Title,Description,Links,Publication Date
1,Глава Минтранса назвал сроки запуска движения ...,Глава Минтранса России Виталий Савельев сообщи...,https://lenta.ru/news/2022/10/08/srokizapuska/,"Sat, 08 Oct 2022 15:37:00 +0300"
2,Пять поездов из Крыма и Севастополя пообещали ...,"Оператор пассажирских перевозок в Крым, компан...",https://lenta.ru/news/2022/10/08/sevas/,"Sat, 08 Oct 2022 15:35:00 +0300"
11,Названы сроки возобновления работы на Керченск...,Керченская паромная переправа возобновит перев...,https://lenta.ru/news/2022/10/08/pereprava/,"Sat, 08 Oct 2022 15:06:00 +0300"
12,Пассажиров отмененных поездов в Крым разместят...,В пресс-службе «Российских железных дорог» соо...,https://lenta.ru/news/2022/10/08/rzd/,"Sat, 08 Oct 2022 15:05:00 +0300"
13,МИД России прокомментировал поздравление Эстон...,МИД России прокомментировал заявления министра...,https://lenta.ru/news/2022/10/08/78437664/,"Sat, 08 Oct 2022 14:58:00 +0300"
...,...,...,...,...
1074,Обломки и повреждения: Следком публиковал виде...,"Следственный комитет России опубликовал видео,...",https://www.mk.ru/incident/2022/10/08/oblomki-...,"Sat, 08 Oct 2022 14:35:30 +0300"
1081,Тамбовчане временно не смогут купить билеты из...,"Кроме того, отменены все поезда на ближайшее в...",https://tambov.mk.ru/social/2022/10/08/tambovc...,"Sat, 08 Oct 2022 14:27:59 +0300"
1083,Поезда «Петербург — Симферополь» задерживаются...,Пассажирские поезда «Петербург — Симферополь» ...,https://www.mk-lenobl.ru/social/2022/10/08/poe...,"Sat, 08 Oct 2022 14:25:30 +0300"
1084,Политолог Сатановский призвал действовать всер...,После теракта на Крымском мосту Кремль не долж...,https://www.mk.ru/incident/2022/10/08/politolo...,"Sat, 08 Oct 2022 14:24:58 +0300"


In [33]:
#выделяю все новости в отдельный датафрейм для лемматизации
df = pd.DataFrame(write_all_news(f_all_news))

### Предобработка текста

In [34]:
df = df.drop_duplicates()

df.shape

(926, 4)

In [35]:
df['Publication Date'] = df['Publication Date'].apply(pd.to_datetime)

In [36]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()
preprocess = lambda text: ' '.join(tokenizer.tokenize(text.lower()))

df['Title'] = df['Title'].apply(preprocess)

In [37]:
from nltk.corpus import stopwords 
  
stop_words = set(stopwords.words('russian')) 

def delete_stop_words(text):

    filtered_tokens = []
    for token in text.split():
        if token not in stop_words:
            filtered_tokens.append(token)
    
    return ' '.join(filtered_tokens)

df['Title'] = df['Title'].apply(delete_stop_words)

In [38]:
from pymorphy2 import MorphAnalyzer

lemmatize = lambda text: ' '.join([MorphAnalyzer().parse(word)[0].normal_form for word in text.split()])

df['Title'] = df['Title'].apply(lemmatize)

### CountVectorizer

In [79]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,3))

In [80]:
def get_mean_cnt(corpus, vectorizer):

    X = vectorizer.fit_transform(corpus['Title']).toarray()
    
    cnts = [30, 50, 100, 150]
    
    result = []
    for count in cnts:
        
        new_freq = np.asarray(X[:count].sum(axis=0)).ravel() / len(X[:count])
        old_freq = np.asarray(X[count:].sum(axis=0)).ravel() / len(X[count:])
    
        razn = np.array(new_freq - old_freq)
        vocab = np.array(vectorizer.get_feature_names_out())
        
        result.append((razn,vocab))
        
    return result

In [81]:
result = get_mean_cnt(df, vectorizer)

In [82]:
for count in result:

    print(count[1][np.argsort(count[0])[-10:]])

['минобороны заявить' 'поезд' 'заявить' 'минобороны' 'украинский'
 'уничтожить' 'российский' 'мост' 'крымский' 'крымский мост']
['назвать' 'российский' 'рассказать' 'взрыв' 'чп' 'взрыв крымский'
 'взрыв крымский мост' 'мост' 'крымский' 'крымский мост']
['пожар крымский' 'пожар крымский мост' 'чп крымский мост' 'чп крымский'
 'чп' 'пожар' 'назвать' 'крымский' 'крымский мост' 'мост']
['из' 'из за' 'за' 'сша' 'россия' 'назвать' 'пожар' 'мост' 'крымский'
 'крымский мост']
